# Task A.1 — Clean & Join

In [ ]:
import pandas as pd
import numpy as np

# Load data
df1 = pd.read_excel("supplier_data1.xlsx")
df2 = pd.read_excel("supplier_data2.xlsx")


# Standardize column names
def clean_columns(df):
    return (df.columns.str.strip()
              .str.lower()
              .str.replace(' ', '_')
              .str.replace('/', '_')
              .str.replace('(', '', regex=False)
              .str.replace(')', '', regex=False))

df1.columns = clean_columns(df1)
df2.columns = clean_columns(df2)

#-----------------------------------------------

# Handle df1
# Replace 0 with NA for mechanical properties
mech_cols = ["rp02", "rm", "ag", "ai"]
for col in mech_cols:
    if col in df1.columns:
        df1[col] = df1[col].replace(0, pd.NA)

# Convert numeric columns
numeric_cols_df1 = ['thickness_mm', 'width_mm', 'gross_weight_kg', 'rp02', 'rm', 'ag', 'ai', 'quantity']
df1[numeric_cols_df1] = df1[numeric_cols_df1].apply(pd.to_numeric, errors='coerce')

# Standardize categorical columns
if 'quality_choice' in df1.columns:
    df1['quality_choice'] = df1['quality_choice'].str.lower().str.strip()

# Translate German terms to English for standardization
if 'finish' in df1.columns:
    finish_map = {
        'gebeizt und geglüht': 'pickled and annealed',
        'ungebeizt': 'unpickled',
        'gebeizt': 'pickled'
    }
    df1['finish'] = df1['finish'].map(finish_map).fillna(df1['finish'])

if 'description' in df1.columns:
    description_map = {
        'Längs- oder Querisse': 'Longitudinal or transverse cracks',
        'Kantenfehler - FS-Kantenrisse': 'Edge errors - FS edge cracks',
        'Sollmasse (Gewicht) unterschritten': 'Target mass (weight) undershot'
    }
    df1['description'] = df1['description'].map(description_map).fillna(df1['description'])

# Add source
df1['data_source'] = 'supplier_data1'

#-----------------------------------------------

# Handle df2
# Convert numeric columns
numeric_cols_df2 = ['weight_kg', 'quantity']
df2[numeric_cols_df2] = df2[numeric_cols_df2].apply(pd.to_numeric, errors='coerce')

# Standardize reserved status
if 'reserved' in df2.columns:
    df2['reserved'] = df2['reserved'].str.strip().str.upper()

# Clean material description
if 'description' in df2.columns:
    df2['material_description'] = df2['description'].str.replace(r'^Material is ', '', regex=True)
    df2.drop('description', axis=1, inplace=True)

# Add source
df2['data_source'] = 'supplier_data2'

#-----------------------------------------------

# Create common schema
df1_common = df1[[
    'quality_choice', 'grade', 'finish', 'thickness_mm', 'width_mm', 
    'description', 'gross_weight_kg', 'quantity', 'rp02', 'rm', 'ag', 'ai', 'data_source'
]].rename(columns={'gross_weight_kg': 'weight_kg'})

df2_common = df2[[
    'material', 'material_description', 'article_id', 'weight_kg', 
    'quantity', 'reserved', 'data_source'
]]

# Add missing columns for unified schema
for col in ['material', 'material_description', 'article_id', 'reserved']:
    if col not in df1_common.columns:
        df1_common.loc[:, col] = pd.NA

for col in ['quality_choice', 'grade', 'finish', 'thickness_mm', 'width_mm', 'description', 'rp02', 'rm', 'ag', 'ai']:
    if col not in df2_common.columns:
        df2_common.loc[:, col] = pd.NA

# Column order before concatenation
all_columns = [
    'article_id', 'quality_choice', 'grade', 'material', 'finish', 'material_description',
    'thickness_mm', 'width_mm', 'description', 'weight_kg', 'quantity', 'reserved',
    'rp02', 'rm', 'ag', 'ai', 'data_source'
]

df1_common = df1_common.reindex(columns=all_columns)
df2_common = df2_common.reindex(columns=all_columns)

#-----------------------------------------------

# Concatenate datasets
inventory_dataset = pd.concat([df1_common, df2_common], ignore_index=True)

# Download csv file
inventory_dataset.to_csv('inventory_dataset.csv', index=False)

# Task B.1 — Reference join & missing values

In [ ]:
# Imports and paths
import pandas as pd
import numpy as np
import re
from pathlib import Path


RFQ_PATH = 'rfq.csv'
REF_PATH = 'reference_properties.tsv'
OUT_ENRICHED_PATH = 'rfq_enriched.csv'
OUT_REPORT_PATH = 'task_b1_join_report.json'

pd.set_option('display.max_columns', 120)



In [ ]:

# Basic alias dictionary derived from observed data
GRADE_ALIASES = {
    's235': 's235jr',
    's355': 's355',  
    'dx51': 'dx51d',
    'cr440y780t dp': 'cr440y780t-dp',
    'cr440y780t-dp': 'cr440y780t-dp',
}

# Suffix families to strip for backoff
SUFFIXES_BACKOFF = [
    '+n', 'jr', 'j2', 'c', 'd', 'gd', 'mc', 'la', 'bh', 'if', 'gd+z'
]

WHITESPACE_SEP_RE = re.compile(r'[\s\-_/]+')
RANGE_SEP_RE = re.compile(r'\s*(?:to|–|-|—|~)\s*')


def normalize_grade(raw: str) -> str:
    if pd.isna(raw):
        return np.nan
    s = str(raw).strip().lower()
    s = WHITESPACE_SEP_RE.sub(' ', s)
    s = s.replace('*', '').strip()
    # unify plus notation like "+N"
    s = s.replace(' + n', '+n').replace(' +n', '+n')
    # apply explicit alias map first
    if s in GRADE_ALIASES:
        return GRADE_ALIASES[s]
    # keep alphanumerics and plus signs/spaces
    return s


def base_grade_for_backoff(grade_norm: str) -> str:
    if pd.isna(grade_norm) or not isinstance(grade_norm, str):
        return grade_norm
    tokens = grade_norm.split()
    # remove simple suffix tokens at end
    tokens = [t for t in tokens if t not in SUFFIXES_BACKOFF]
    s = ' '.join(tokens)
    # remove trailing suffixes attached without space (e.g., s355jr, s355j2)
    s = re.sub(r'(s\d{3})(?:jr|j2|c)$', r'\1', s)
    s = re.sub(r'(dx\d{2})d$', r'\1d', s)  # dx51d stays dx51d
    return s


def parse_range_pair(min_val, max_val):
    """Parse two fields possibly as strings; returns (min, max, mid).
    If one side missing but the other present -> mirror. If min>max -> swap.
    """
    def to_float(x):
        if pd.isna(x) or x == '':
            return np.nan
        try:
            return float(str(x).strip())
        except Exception:
            # attempt split within a single cell like '4.9-5.1'
            parts = RANGE_SEP_RE.split(str(x))
            if len(parts) == 2:
                try:
                    a = float(parts[0]); b = float(parts[1])
                    return (a + b) / 2.0
                except Exception:
                    return np.nan
            return np.nan
    a = to_float(min_val)
    b = to_float(max_val)
    if pd.isna(a) and pd.isna(b):
        return np.nan, np.nan, np.nan
    if pd.isna(a) and not pd.isna(b):
        a = b
    if pd.isna(b) and not pd.isna(a):
        b = a
    if not pd.isna(a) and not pd.isna(b) and a > b:
        a, b = b, a
    mid = np.nan if pd.isna(a) or pd.isna(b) else (a + b) / 2.0
    return a, b, mid


def parse_ref_range_str(s: str):
    """Parse strings like '470-630 MPa' -> (470, 630)."""
    if pd.isna(s):
        return np.nan, np.nan
    txt = str(s)
    # extract numbers
    nums = re.findall(r'[-+]?[0-9]*\.?[0-9]+', txt)
    if len(nums) == 0:
        return np.nan, np.nan
    if len(nums) == 1:
        v = float(nums[0])
        return v, v
    return float(nums[0]), float(nums[1])



In [ ]:
# Load data
rfq = pd.read_csv(RFQ_PATH)
ref = pd.read_csv(REF_PATH, sep='\t')

# Keep original grade
rfq['grade_original'] = rfq['grade']

# Normalize grades
rfq['grade_normalized'] = rfq['grade'].apply(normalize_grade)
ref['grade_original_ref'] = ref['Grade/Material']
ref['grade_normalized'] = ref['Grade/Material'].apply(normalize_grade)

# Base grade for backoff
rfq['grade_base'] = rfq['grade_normalized'].apply(base_grade_for_backoff)
ref['grade_base'] = ref['grade_normalized'].apply(base_grade_for_backoff)

rfq.head(3), ref.head(3)[['grade_original_ref','grade_normalized','grade_base']]



(                                     id   grade  grade_suffix coating  \
 0  8aff426d-b8c0-43aa-ad26-835ef4de6129  S700MC           NaN     NaN   
 1  37e624be-b125-464f-85b6-1838530193ef  S250GD           NaN   ZM310   
 2  b8257184-6307-46ab-b06e-d979336d1263   DX51D           NaN    Z100   
 
                          finish surface_type surface_protection        form  \
 0                         Oiled          NaN                NaN       Coils   
 1  Hot-dip zinc magnesium (+ZM)          NaN                NaN  Slit Coils   
 2   Hot-dip Galvanized (+Z/+GI)          NaN  Lightly Oiled (L)       Coils   
 
    thickness_min  thickness_max  width_min  width_max  length_min  height_min  \
 0            6.0            6.0      600.0     1520.0         NaN         NaN   
 1            1.5            1.5      327.0      327.0         NaN         NaN   
 2            0.4            0.4     1000.0     1500.0         NaN         NaN   
 
    height_max  weight_min  weight_max  inner_diam

In [ ]:
# Parse RFQ numeric ranges

def add_range_cols(df: pd.DataFrame, col_prefix: str):
    a, b, m = [], [], []
    min_col, max_col = f'{col_prefix}_min', f'{col_prefix}_max'
    for i in range(len(df)):
        mn, mx, md = parse_range_pair(df.at[i, min_col] if min_col in df.columns else np.nan,
                                      df.at[i, max_col] if max_col in df.columns else np.nan)
        a.append(mn); b.append(mx); m.append(md)
    df[f'{col_prefix}_min'] = a
    df[f'{col_prefix}_max'] = b
    df[f'{col_prefix}_mid'] = m

range_prefixes = [
    'thickness', 'width', 'height', 'weight', 'inner_diameter', 'outer_diameter', 'yield_strength', 'tensile_strength'
]

for p in range_prefixes:
    add_range_cols(rfq, p)

# Single-sided values: length_min only present in sample; derive mid and max if max missing
if 'length_min' in rfq.columns:
    mn = pd.to_numeric(rfq['length_min'], errors='coerce')
    mx = pd.to_numeric(rfq.get('length_max', np.nan), errors='coerce') if 'length_max' in rfq.columns else np.nan
    if isinstance(mx, pd.Series):
        # fill mirror
        mx = mx.fillna(mn)
    rfq['length_min'] = mn
    rfq['length_max'] = mx if isinstance(mx, pd.Series) else mn
    rfq['length_mid'] = (rfq['length_min'] + rfq['length_max']) / 2.0

# Form-driven flags
rfq['form_norm'] = rfq.get('form', '').fillna('').str.strip().str.lower()
rfq['has_thickness'] = rfq['thickness_min'].notna()
rfq['has_width'] = rfq['width_min'].notna()
rfq['has_length'] = rfq['length_min'].notna()
rfq['has_id_od'] = rfq['inner_diameter_min'].notna() | rfq['outer_diameter_min'].notna()



In [ ]:
# Prepare reference: parse mechanical ranges
ref[['ref_tensile_min','ref_tensile_max']] = ref['Tensile strength (Rm)'].apply(lambda s: pd.Series(parse_ref_range_str(s)))
ref[['ref_yield_min','ref_yield_max']] = ref['Yield strength (Re or Rp0.2)'].apply(lambda s: pd.Series(parse_ref_range_str(s)))

ref_simple = ref[['grade_normalized','grade_base','ref_tensile_min','ref_tensile_max','ref_yield_min','ref_yield_max','Category','Standards']].copy()

# Resolve duplicates in reference: keep row with widest tensile window, then widest yield
def pick_best_ref(group: pd.DataFrame) -> pd.Series:
    g = group.copy()
    g['ten_width'] = (g['ref_tensile_max'] - g['ref_tensile_min']).abs()
    g['yld_width'] = (g['ref_yield_max'] - g['ref_yield_min']).abs()
    g = g.sort_values(['ten_width','yld_width'], ascending=[False, False])
    return g.iloc[0]

ref_best_exact = ref_simple.groupby('grade_normalized', as_index=False).apply(pick_best_ref).reset_index(drop=True)
ref_best_base = ref_simple.groupby('grade_base', as_index=False).apply(pick_best_ref).reset_index(drop=True)

len(ref_best_exact), len(ref_best_base)



In [ ]:
# Join RFQ with reference: exact then backoff

# Exact match join
rfq_enriched = rfq.merge(ref_best_exact.add_prefix('exact_'), left_on='grade_normalized', right_on='exact_grade_normalized', how='left')

# Backoff join for rows without exact match
mask_no_exact = rfq_enriched['exact_grade_normalized'].isna()
rfq_backoff = rfq.loc[mask_no_exact, ['id','grade_base']].merge(
    ref_best_base.add_prefix('base_'), left_on='grade_base', right_on='base_grade_base', how='left'
)
rfq_enriched = rfq_enriched.merge(rfq_backoff[['id','base_grade_base','base_ref_tensile_min','base_ref_tensile_max','base_ref_yield_min','base_ref_yield_max','base_Category','base_Standards']], on='id', how='left')

# Compose reference fields and join status
for tgt, ex, bf in [
    ('ref_tensile_min','exact_ref_tensile_min','base_ref_tensile_min'),
    ('ref_tensile_max','exact_ref_tensile_max','base_ref_tensile_max'),
    ('ref_yield_min','exact_ref_yield_min','base_ref_yield_min'),
    ('ref_yield_max','exact_ref_yield_max','base_ref_yield_max'),
    ('ref_category','exact_Category','base_Category'),
    ('ref_standards','exact_Standards','base_Standards'),
]:
    rfq_enriched[tgt] = rfq_enriched[ex].combine_first(rfq_enriched[bf])

rfq_enriched['has_ref_match'] = rfq_enriched['ref_tensile_min'].notna() | rfq_enriched['ref_yield_min'].notna()
rfq_enriched['ref_join_status'] = np.where(
    rfq_enriched['exact_grade_normalized'].notna(), 'exact_match',
    np.where(rfq_enriched['base_grade_base'].notna(), 'family_backoff', 'no_match')
)

rfq_enriched.head(3)[['id','grade','grade_normalized','grade_base','ref_join_status','ref_tensile_min','ref_tensile_max','ref_yield_min','ref_yield_max']]



,id,grade,grade_normalized,grade_base,ref_join_status,ref_tensile_min,ref_tensile_max,ref_yield_min,ref_yield_max
0,8aff426d-b8c0-43aa-ad26-835ef4de6129,S700MC,s700mc,s700mc,exact_match,750.0,-950.0,700.0,700.0
1,37e624be-b125-464f-85b6-1838530193ef,S250GD,s250gd,s250gd,exact_match,330.0,-510.0,250.0,250.0
2,b8257184-6307-46ab-b06e-d979336d1263,DX51D,dx51d,dx51d,exact_match,270.0,-500.0,350.0,350.0


In [ ]:
# Impute missing mechanicals from reference

for tgt_min, tgt_max, ref_min, ref_max in [
    ('yield_strength_min','yield_strength_max','ref_yield_min','ref_yield_max'),
    ('tensile_strength_min','tensile_strength_max','ref_tensile_min','ref_tensile_max'),
]:
    need_min = rfq_enriched[tgt_min].isna() & rfq_enriched[ref_min].notna()
    need_max = rfq_enriched[tgt_max].isna() & rfq_enriched[ref_max].notna()
    rfq_enriched.loc[need_min, tgt_min] = rfq_enriched.loc[need_min, ref_min]
    rfq_enriched.loc[need_max, tgt_max] = rfq_enriched.loc[need_max, ref_max]

rfq_enriched['is_mech_imputed'] = (
    (rfq_enriched['yield_strength_min'].isna() & rfq_enriched['ref_yield_min'].notna()) |
    (rfq_enriched['yield_strength_max'].isna() & rfq_enriched['ref_yield_max'].notna()) |
    (rfq_enriched['tensile_strength_min'].isna() & rfq_enriched['ref_tensile_min'].notna()) |
    (rfq_enriched['tensile_strength_max'].isna() & rfq_enriched['ref_tensile_max'].notna())
).astype(int)

# Recompute mids for mechanicals
for p in ['yield_strength','tensile_strength']:
    a = rfq_enriched[f'{p}_min']
    b = rfq_enriched[f'{p}_max']
    rfq_enriched[f'{p}_mid'] = np.where(a.notna() & b.notna(), (a + b) / 2.0, np.nan)

rfq_enriched[['id','ref_join_status','yield_strength_min','yield_strength_max','tensile_strength_min','tensile_strength_max','is_mech_imputed']].head(3)



,id,ref_join_status,yield_strength_min,yield_strength_max,tensile_strength_min,tensile_strength_max,is_mech_imputed
0,8aff426d-b8c0-43aa-ad26-835ef4de6129,exact_match,700.0,700.0,760.0,810.0,0
1,37e624be-b125-464f-85b6-1838530193ef,exact_match,250.0,250.0,330.0,-510.0,0
2,b8257184-6307-46ab-b06e-d979336d1263,exact_match,350.0,350.0,270.0,-500.0,0


In [ ]:
# Categorical normalization and flags
for col in ['form','coating','finish','surface_type','surface_protection']:
    if col in rfq_enriched.columns:
        rfq_enriched[col] = rfq_enriched[col].astype(str).str.strip().replace({'': np.nan})

rfq_enriched['has_dim_data'] = rfq_enriched[['has_thickness','has_width','has_length','has_id_od']].any(axis=1)

rfq_enriched.head(2)



,id,grade,grade_suffix,coating,finish,surface_type,surface_protection,form,thickness_min,thickness_max,width_min,width_max,length_min,height_min,height_max,weight_min,weight_max,inner_diameter_min,inner_diameter_max,outer_diameter_min,outer_diameter_max,yield_strength_min,yield_strength_max,tensile_strength_min,tensile_strength_max,grade_original,grade_normalized,grade_base,thickness_mid,width_mid,height_mid,weight_mid,inner_diameter_mid,outer_diameter_mid,yield_strength_mid,tensile_strength_mid,length_max,length_mid,form_norm,has_thickness,has_width,has_length,has_id_od,exact_grade_normalized,exact_grade_base,exact_ref_tensile_min,exact_ref_tensile_max,exact_ref_yield_min,exact_ref_yield_max,exact_Category,exact_Standards,exact_ten_width,exact_yld_width,base_grade_base,base_ref_tensile_min,base_ref_tensile_max,base_ref_yield_min,base_ref_yield_max,base_Category,base_Standards,ref_tensile_min,ref_tensile_max,ref_yield_min,ref_yield_max,ref_category,ref_standards,has_ref_match,ref_join_status,is_mech_imputed,has_dim_data
0,8aff426d-b8c0-43aa-ad26-835ef4de6129,S700MC,NaN,nan,Oiled,nan,nan,Coils,6.0,6.0,600.0,1520.0,NaN,NaN,NaN,15000.0,25000.0,610.0,610.0,NaN,NaN,700.0,700.0,760.0,810.0,S700MC,s700mc,s700mc,6.0,1060.0,NaN,20000.0,610.0,NaN,700.0,785.0,NaN,NaN,coils,True,True,False,True,s700mc,s700mc,750.0,-950.0,700.0,700.0,Microalloyed Steel,EN 10149-2:2013,1700.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,750.0,-950.0,700.0,700.0,Microalloyed Steel,EN 10149-2:2013,True,exact_match,0,True
1,37e624be-b125-464f-85b6-1838530193ef,S250GD,NaN,ZM310,Hot-dip zinc magnesium (+ZM),nan,nan,Slit Coils,1.5,1.5,327.0,327.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,250.0,250.0,330.0,-510.0,S250GD,s250gd,s250gd,1.5,327.0,NaN,NaN,NaN,NaN,250.0,-90.0,NaN,NaN,slit coils,True,True,False,False,s250gd,s250gd,330.0,-510.0,250.0,250.0,Galvanized Steel,EN 10346:2015,840.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,330.0,-510.0,250.0,250.0,Galvanized Steel,EN 10346:2015,True,exact_match,0,True


In [ ]:
# Report and save
report = {
    'counts_by_ref_join_status': rfq_enriched['ref_join_status'].value_counts(dropna=False).to_dict(),
    'num_with_ref_match': int(rfq_enriched['has_ref_match'].sum()),
    'pct_with_ref_match': float((rfq_enriched['has_ref_match'].mean() * 100).round(2)),
    'pct_mech_imputed': float((rfq_enriched['is_mech_imputed'].mean() * 100).round(2)),
    'top_unmapped_grades': rfq_enriched.loc[rfq_enriched['ref_join_status']=='no_match','grade'].value_counts().head(20).to_dict(),
}

rfq_enriched.to_csv(OUT_ENRICHED_PATH, index=False)

import json
with open(OUT_REPORT_PATH, 'w', encoding='utf-8') as f:
    json.dump(report, f, ensure_ascii=False, indent=2)

report


{'counts_by_ref_join_status': {'exact_match': 941, 'no_match': 59},
 'num_with_ref_match': 940,
 'pct_with_ref_match': 94.0,
 'pct_mech_imputed': 0.0,
 'top_unmapped_grades': {}}

# Task B.2 — Feature engineering

In [ ]:

# Interval IoU for 1D ranges

def interval_iou(a_min, a_max, b_min, b_max):
    if any(pd.isna(v) for v in [a_min, a_max, b_min, b_max]):
        return np.nan
    # ensure order
    if a_min > a_max:
        a_min, a_max = a_max, a_min
    if b_min > b_max:
        b_min, b_max = b_max, b_min
    inter = max(0.0, min(a_max, b_max) - max(a_min, b_min))
    ua = max(0.0, a_max - a_min)
    ub = max(0.0, b_max - b_min)
    union = ua + ub - inter
    if union == 0.0:
        # both singletons: equal -> 1 else 0
        return 1.0 if a_min == b_min == a_max == b_max else 0.0
    return inter / union

# Categorical exact match (1/0), missing -> 0

def cat_match(a, b):
    if pd.isna(a) or pd.isna(b):
        return 0.0
    return 1.0 if str(a).strip().lower() == str(b).strip().lower() else 0.0

# Mechanical similarity using exponential decay of absolute difference

def mech_sim(x, y, scale):
    if pd.isna(x) or pd.isna(y):
        return np.nan
    return float(np.exp(-abs(float(x) - float(y)) / float(scale)))

# Select dimension fields by form family

SHEET_FORMS = {'coils','coil','sheets','sheet','plates','plate','strips','strip','equal angles','rectangular tubes'}
TUBE_FORMS = {'round tubes','round tube','tubes','tube'}


def form_family(form_value: str) -> str:
    if pd.isna(form_value):
        return 'unknown'
    s = str(form_value).strip().lower()
    if s in TUBE_FORMS:
        return 'tube'
    if s in SHEET_FORMS:
        return 'sheet'
    return 'unknown'

DIM_FIELDS_BY_FAMILY = {
    'sheet': [('thickness_min','thickness_max'), ('width_min','width_max'), ('length_min','length_max')],
    'tube': [('inner_diameter_min','inner_diameter_max'), ('outer_diameter_min','outer_diameter_max'), ('thickness_min','thickness_max')],
    'unknown': [('thickness_min','thickness_max'), ('width_min','width_max')]
}

# Per-pair feature computation

def compute_pair_features(a: pd.Series, b: pd.Series, scales=(50.0, 75.0)):
    fam = form_family(a.get('form', np.nan))
    dim_pairs = DIM_FIELDS_BY_FAMILY[fam]
    dim_ious = []
    for mn, mx in dim_pairs:
        sim = interval_iou(a.get(mn), a.get(mx), b.get(mn), b.get(mx))
        if not pd.isna(sim):
            dim_ious.append(sim)
    dim_sim = np.nan if len(dim_ious) == 0 else float(np.mean(dim_ious))

    # Categoricals
    coat_sim = cat_match(a.get('coating'), b.get('coating'))
    finish_sim = cat_match(a.get('finish'), b.get('finish'))
    stype_sim = cat_match(a.get('surface_type'), b.get('surface_type'))
    form_sim = cat_match(a.get('form'), b.get('form'))
    cat_sims = [coat_sim, finish_sim, stype_sim]
    cat_sim = float(np.mean(cat_sims)) if len(cat_sims) else np.nan

    # Mechanicals
    y_sim = mech_sim(a.get('yield_strength_mid'), b.get('yield_strength_mid'), scales[0])
    t_sim = mech_sim(a.get('tensile_strength_mid'), b.get('tensile_strength_mid'), scales[1])
    mech_vals = [v for v in [y_sim, t_sim] if not pd.isna(v)]
    mech_sim_avg = np.nan if len(mech_vals) == 0 else float(np.mean(mech_vals))

    return {
        'form_match': form_sim,
        'dim_sim': dim_sim,
        'coat_match': coat_sim,
        'finish_match': finish_sim,
        'surface_type_match': stype_sim,
        'cat_sim': cat_sim,
        'yield_sim': y_sim,
        'tensile_sim': t_sim,
        'mech_sim': mech_sim_avg,
    }

# Aggregate into overall similarity with renormalized weights

def aggregate_similarity(feats: dict, weights=None):
    if weights is None:
        weights = {'dim_sim': 0.5, 'cat_sim': 0.2, 'mech_sim': 0.3}
    parts = []
    wts = []
    for key, w in weights.items():
        v = feats.get(key, np.nan)
        if not pd.isna(v):
            parts.append(v * w)
            wts.append(w)
    if len(parts) == 0:
        return np.nan
    return float(sum(parts) / sum(wts))



# Task B.3

In [ ]:
# Similarity calculation and Top-3 per RFQ

# Define an aggregate scorer using the utilities from B.2
DEFAULT_WEIGHTS = {'dim_sim': 0.5, 'cat_sim': 0.2, 'mech_sim': 0.3}
DEFAULT_SCALES = (50.0, 75.0)  # (yield, tensile) MPa

# Define a signature to identify exact duplicates (to exclude)
SIG_COLS = [
    'grade_normalized','form','coating','finish','surface_type',
    'thickness_min','thickness_max','width_min','width_max','length_min','length_max',
    'inner_diameter_min','inner_diameter_max','outer_diameter_min','outer_diameter_max',
    'yield_strength_mid','tensile_strength_mid'
]


def make_signature(row: pd.Series) -> tuple:
    return tuple((row.get(c) if c in row.index else np.nan) for c in SIG_COLS)


# Precompute signatures and an index by form to block comparisons
work = rfq_enriched.copy()
work['form_norm'] = work['form'].astype(str).str.strip().str.lower()
work['signature'] = work.apply(make_signature, axis=1)

# Build a dict: form -> dataframe of rows in that form
form_groups = {k: v.copy() for k, v in work.groupby('form_norm', dropna=False)}


def topk_for_query(query_row: pd.Series, candidates_df: pd.DataFrame, k=3,
                   weights=DEFAULT_WEIGHTS, scales=DEFAULT_SCALES):
    results = []
    for _, cand in candidates_df.iterrows():
        if cand['id'] == query_row['id']:
            continue  # exclude self
        if cand['signature'] == query_row['signature']:
            continue  # exclude exact duplicate
        feats = compute_pair_features(query_row, cand, scales=scales)
        if feats.get('form_match', 0.0) < 1.0:
            # enforce hard filter on form match
            continue
        sim = aggregate_similarity(feats, weights=weights)
        if pd.isna(sim):
            continue
        results.append({
            'query_id': query_row['id'],
            'neighbor_id': cand['id'],
            'similarity': float(sim),
            **feats,
        })
    if not results:
        return []
    df_res = pd.DataFrame(results).sort_values('similarity', ascending=False).head(k)
    df_res['rank'] = np.arange(1, len(df_res) + 1)
    return df_res.to_dict(orient='records')


# Compute top-3 per RFQ within the same form block
all_top3 = []
for form_key, df_block in form_groups.items():
    for _, row in df_block.iterrows():
        recs = topk_for_query(row, df_block, k=3)
        all_top3.extend(recs)

Top3_df = pd.DataFrame(all_top3)
Top3_df.head(10)



,query_id,neighbor_id,similarity,form_match,dim_sim,coat_match,finish_match,surface_type_match,cat_sim,yield_sim,tensile_sim,mech_sim,rank
0,8aff426d-b8c0-43aa-ad26-835ef4de6129,7d1ab305-7fc6-4ab0-bc2a-9ae1e038e67e,0.930616,1.0,0.994565,1.0,0.0,1.0,0.666667,1.0,1.000000,1.000000,1
1,8aff426d-b8c0-43aa-ad26-835ef4de6129,2624655e-ea07-468a-8da8-2e39c9d1e7f0,0.716066,1.0,0.494565,1.0,1.0,1.0,1.000000,1.0,0.791890,0.895945,2
2,8aff426d-b8c0-43aa-ad26-835ef4de6129,2e56f82b-a80d-4704-83a5-14a52a117b65,0.666667,1.0,NaN,1.0,0.0,1.0,0.666667,NaN,NaN,NaN,3
3,b8257184-6307-46ab-b06e-d979336d1263,5ad0afbf-dc39-4adb-bbd5-c18012316bb5,0.754793,1.0,NaN,1.0,0.0,1.0,0.666667,1.0,0.627089,0.813545,1
4,b8257184-6307-46ab-b06e-d979336d1263,0dcb40f1-38f9-44d1-a023-99d80f143a79,0.733333,1.0,NaN,0.0,0.0,1.0,0.333333,1.0,1.000000,1.000000,2
5,b8257184-6307-46ab-b06e-d979336d1263,0810e19e-0170-4a1e-930b-f5830e698dce,0.733333,1.0,NaN,0.0,0.0,1.0,0.333333,1.0,1.000000,1.000000,3
6,84be5122-71d5-4a35-a912-fd2911e3c64f,02647f9b-5108-4955-938f-e6b003b62690,1.000000,1.0,1.000000,1.0,1.0,1.0,1.000000,NaN,NaN,NaN,1
7,84be5122-71d5-4a35-a912-fd2911e3c64f,2e56f82b-a80d-4704-83a5-14a52a117b65,1.000000,1.0,NaN,1.0,1.0,1.0,1.000000,NaN,NaN,NaN,2
8,84be5122-71d5-4a35-a912-fd2911e3c64f,81c9dcd6-b91d-4a3c-a6a8-5a72d0a7d68c,1.000000,1.0,1.000000,1.0,1.0,1.0,1.000000,NaN,NaN,NaN,3
9,0f595b0b-2253-4585-a672-fda592da7191,b954d3d2-f2d8-4b11-955e-31c7839517f5,0.933333,1.0,1.000000,1.0,0.0,1.0,0.666667,1.0,1.000000,1.000000,1


In [ ]:
# Save Top-3 output
TOP3_PATH = 'rfq_top3_similarity.csv'

cols = ['query_id','neighbor_id','rank','similarity','form_match','dim_sim','cat_sim','mech_sim','coat_match','finish_match','surface_type_match','yield_sim','tensile_sim']
Top3_df[cols].to_csv(TOP3_PATH, index=False)
TOP3_PATH, len(Top3_df)



('rfq_top3_similarity.csv', 2979)

In [ ]:
# Demo: compute similarity with explanations for a small sample

# pick a few RFQs with non-null form
sample_ids = rfq_enriched[rfq_enriched['form'].notna()]['id'].head(3).tolist()

pairs = []
for i in range(min(2, len(sample_ids))):
    for j in range(i+1, min(3, len(sample_ids))):
        ida, idb = sample_ids[i], sample_ids[j]
        a = rfq_enriched.set_index('id').loc[ida]
        b = rfq_enriched.set_index('id').loc[idb]
        feats = compute_pair_features(a, b)
        sim = aggregate_similarity(feats)
        pairs.append({
            'id_a': ida,
            'id_b': idb,
            **feats,
            'similarity': sim,
        })

pd.DataFrame(pairs)



,id_a,id_b,form_match,dim_sim,coat_match,finish_match,surface_type_match,cat_sim,yield_sim,tensile_sim,mech_sim,similarity
0,8aff426d-b8c0-43aa-ad26-835ef4de6129,37e624be-b125-464f-85b6-1838530193ef,0.0,0.000000,0.0,0.0,1.0,0.333333,0.000123,0.000009,0.000066,0.066686
1,8aff426d-b8c0-43aa-ad26-835ef4de6129,b8257184-6307-46ab-b06e-d979336d1263,1.0,0.271739,0.0,0.0,1.0,0.333333,0.000912,0.000006,0.000459,0.202674
2,37e624be-b125-464f-85b6-1838530193ef,b8257184-6307-46ab-b06e-d979336d1263,0.0,0.000000,0.0,0.0,1.0,0.333333,0.135335,0.716531,0.425933,0.194447
